In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

file_path = '/Users/ks/exploratory_data_analysis/output_data/original_output_data.csv'
transformed_data = pd.read_csv(file_path)

## Task 1: Current state of the loans.

transformed_data['percent_recovered_investor'] = (transformed_data['total_rec_prncp'] + transformed_data['total_rec_late_fee']) / transformed_data['funded_amount_inv'] * 100
# Calculate percentage of loans recovered against total amount funded
transformed_data['percent_recovered_total'] = (transformed_data['total_rec_prncp'] + transformed_data['total_rec_late_fee']) / transformed_data['total_payment'] * 100
# Print summary statistics
print(f"Mean percentage recovered against investor funding: {transformed_data['percent_recovered_investor'].mean():.2f}%")
print(f"Mean percentage recovered against total amount funded: {transformed_data['percent_recovered_total'].mean():.2f}%")
# Visualize mean percentages on a bar chart
plt.figure(figsize=(10, 6))
percentages = [
    transformed_data['percent_recovered_investor'].mean(),
    transformed_data['percent_recovered_total'].mean()
]
labels = ['Recovered vs Investor Funding', 'Recovered vs Total Funding']
plt.bar(labels, percentages, color=['blue', 'green'])
plt.ylim(0, 100)
plt.ylabel('Percentage')
plt.title('Mean Percentage of Loans Recovered')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
projected_recovery_6_months = 0.75  # 75% recovery up to 6 months in the future
# Visualize percentage recovered up to 6 months in the future
plt.figure(figsize=(10, 6))
plt.bar(['Up to 6 Months'], [projected_recovery_6_months * 100], color='orange')
plt.ylim(0, 100)
plt.ylabel('Percentage')
plt.title('Projected Percentage of Total Amount Recovered up to 6 Months')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## Task 2. Calculating loss.

charged_off_loans = transformed_data[transformed_data['loan_status'] == 'Charged Off']
# Calculate percentage of charged off loans
charged_off_percentage = (charged_off_loans.shape[0] / transformed_data.shape[0]) * 100
# Calculate total amount paid towards charged off loans
total_amount_paid_charged_off = charged_off_loans['total_payment'].sum()
# Print results
print(f"Percentage of loans charged off: {charged_off_percentage:.2f}%")
print(f"Total amount paid towards charged off loans: ${total_amount_paid_charged_off:.2f}")

## Task 3: Calculating projected loss

# Calculate total funded amount for charged off loans
total_funded_amount = charged_off_loans['funded_amount_inv'].sum()
total_amount_recovered = (charged_off_loans['total_rec_prncp'] + charged_off_loans['total_rec_late_fee']).sum()
projected_loss = total_funded_amount - total_amount_recovered
print(f"Projected loss due to Charged Off loans: ${projected_loss:.2f}")
# Extract numeric term (months) from 'term' column
charged_off_loans['term_months'] = charged_off_loans['term'].str.extract('(\d+)').astype(float)
# Convert issue_date to datetime format with the correct format
charged_off_loans['issue_date'] = pd.to_datetime(charged_off_loans['issue_date'], format='%b-%Y')
# Calculate the remaining term in months
charged_off_loans['remaining_term_months'] = charged_off_loans['term_months'] - charged_off_loans['issue_date'].dt.month
# Filter out negative remaining term months
charged_off_loans = charged_off_loans[charged_off_loans['remaining_term_months'] > 0]
charged_off_loans['projected_loss_per_month'] = projected_loss / charged_off_loans['remaining_term_months']
# Visualize projected loss over remaining term
plt.figure(figsize=(30, 10))
plt.bar(charged_off_loans.index, charged_off_loans['projected_loss_per_month'], color='red')
plt.xlabel('Loans')
plt.ylabel('Projected Loss ($)')
plt.title('Projected Loss Over Remaining Term of Charged Off Loans')
plt.xticks(charged_off_loans.index, charged_off_loans['term'], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Task 4. 

# Create a copy explicitly to avoid messing up the main dataframe and to avoid errors
late_payments = transformed_data[transformed_data['loan_status'].isin(['Late (16-30 days)', 'Late (31-120 days)'])].copy()
# Calculate percentage of customers with late payments
percentage_late_payments = (late_payments.shape[0] / transformed_data.shape[0]) * 100
print(f"Percentage of customers with late payments: {percentage_late_payments:.2f}%")
# Calculate total amount of customers with late payments
total_customers_late_payments = late_payments.shape[0]
print(f"Total number of customers with late payments: {total_customers_late_payments}")
# Calculate potential loss if late payments convert to charged off
total_amount_paid_late_payments = late_payments['total_payment'].sum()
print(f"Total amount paid towards loans with late payments: ${total_amount_paid_late_payments:.2f}")
# Assuming 'charged_off_loss' is the projected loss if all late payments were charged off
charged_off_loss = total_amount_paid_late_payments
print(f"Projected loss if all loans with late payments convert to Charged Off: ${charged_off_loss:.2f}")
# Projected loss if late payments were to finish full term
total_funded_amount_late_payments = late_payments['funded_amount_inv'].sum()
total_amount_recovered_late_payments = (late_payments['total_rec_prncp'] + late_payments['total_rec_late_fee']).sum()
projected_loss_late_payments = total_funded_amount_late_payments - total_amount_recovered_late_payments
print(f"Projected loss if loans with late payments finish their full term: ${projected_loss_late_payments:.2f}")
# Calculate percentage of total expected revenue
total_funded_amount_all_loans = transformed_data['funded_amount_inv'].sum()
percentage_total_expected_revenue_lost = (charged_off_loss + projected_loss_late_payments) / total_funded_amount_all_loans * 100
print(f"Percentage of total expected revenue lost: {percentage_total_expected_revenue_lost:.2f}%")

## Task 5. Indicators of loss

# Subset for charged off loans
charged_off_loans = transformed_data[transformed_data['loan_status'] == 'Charged Off']
# Subset for late payments
late_payments = transformed_data[transformed_data['loan_status'].isin(['Late (16-30 days)', 'Late (31-120 days)'])]

# Function to plot distribution of a categorical variable
def plot_categorical_comparison(data1, data2, column, xlabel):
    categories1 = data1[column].value_counts(normalize=True).sort_index()
    categories2 = data2[column].value_counts(normalize=True).sort_index()

    # Combine unique categories from both datasets
    all_categories = sorted(set(categories1.index) | set(categories2.index))

    # Reindex to ensure both dataframes have the same categories
    categories1 = categories1.reindex(all_categories, fill_value=0)
    categories2 = categories2.reindex(all_categories, fill_value=0)

    fig, ax = plt.subplots(figsize=(10, 6))
    width = 0.35
    x = range(len(all_categories))
    ax.bar(x, categories1, width, label='Charged Off', color='blue', alpha=0.7)
    ax.bar([i + width for i in x], categories2, width, label='Late Payments', color='orange', alpha=0.7)
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Proportion')
    ax.set_title(f'Distribution of {xlabel} between Charged Off and Late Payments')
    ax.set_xticks([i + width / 2 for i in x])
    ax.set_xticklabels(all_categories, rotation=45)
    ax.legend()
    plt.tight_layout()
    plt.show()

# Analyze Loan Grade (grade)
plot_categorical_comparison(charged_off_loans, late_payments, 'grade', 'Loan Grade')

# Analyze Loan Purpose (purpose)
plot_categorical_comparison(charged_off_loans, late_payments, 'purpose', 'Loan Purpose')

# Analyze Home Ownership (home_ownership)
plot_categorical_comparison(charged_off_loans, late_payments, 'home_ownership', 'Home Ownership')
